해당 notebook은 간단한 TPU 예제이다.
아래의 예제는 "초보자를 위한 빠른 시작"에 있는 MNIST model로 부터 출발하였다.
이 예제에는 COLAB TPU 사용에 대하여 거의 모든 부분이 포함되어 있다.

"00_MNIST_beginner_non_TPU.py"와 "01_MNIST_beginner_TPU.py"를 compare 도구를 사용하여 비교해 보기 바란다.


아래의 code 부분은 관용적으로 가지고 다니기 바란다.
아래의 부분은 단순히 tensorflow의 version check 부분이다.
별 의미는 없으니 출력하는 문구 정도만 변경해서 쓰면 된다.

```
import os

import distutils
if distutils.version.LooseVersion(tf.__version__) <= '2.0':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

```



In [7]:
import tensorflow as tf
import numpy as np
import os

import distutils
if distutils.version.LooseVersion(tf.__version__) <= '2.0':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

아래의 부분은 단지 network를 시험하기 위하여 좀 복잡하게 만든것 뿐이다.
TPU 성능 평가를 위하여 만든 것이다. 이정도의 network 크기면 어느 정도 성능 test가 가능하다.

In [8]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model


아래의 부분은 colab에서 TPU의 strategy.scope()를 정의하는 부분이다.
copare 도구를 사용하여, code가 추가된 부분과 "strategy.scope"에 대해서 해당 되는 부분을 유의하여서 보자.
```
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
```



In [9]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=17,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=17
)

model.save_weights('./fashion_mnist.h5', overwrite=True)

INFO:tensorflow:Initializing the TPU system: grpc://10.12.107.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.12.107.250:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Epoch 1/17
 1/60 [..............................] - ETA: 2:43 - loss: 4.5074 - sparse_categorical_accuracy: 0.1030WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0211s). Check your callbacks.


60/60 [==============================] - 4s 68ms/step - loss: 1.2730 - sparse_categorical_accuracy: 0.6567
Epoch 2/17
60/60 [==============================] - 1s 22ms/step - loss: 0.5453 - sparse_categorical_accuracy: 0.8138
Epoch 3/17
60/60 [==============================] - 1s 22ms/step - loss: 0.4444 - sparse_categorical_accuracy: 0.8469
Epoch 4/17
60/60 [==============================] - 1s 23ms/step - loss: 0.3821 - sparse_categorical_accuracy: 0.8663
Epoch 5/17
60/60 [==============================] - 1s 22ms/step - loss: 0.3509 - sparse_categorical_accuracy: 0.8767
Epoch 6/17
60/60 [==============================] - 1s 22ms/step - loss: 0.3118 - sparse_categorical_accuracy: 0.8884
Epoch 7/17
60/60 [==============================] - 1s 22ms/step - loss: 0.2858 - sparse_categorical_accuracy: 0.8984
Epoch 8/17
60/60 [==============================] - 1s 22ms/step - loss: 0.2736 - sparse_categorical_accuracy: 0.9007
Epoch 9/17
60/60 [==============================] - 1s 22ms/step - 

60/60 [==============================] - 7s 122ms/step - loss: 0.1602 - sparse_categorical_accuracy: 0.9393 - val_loss: 0.2212 - val_sparse_categorical_accuracy: 0.9241
